# Shallow GP benchmarks

Shallow GP benchmarks for 'Kernels with Latent Gaussian Processes for Non-stationary Regression' paper.
Here we train and evaluate GPs with the following kernels
- Squared exponemtial
- Squared exponemtial times periodic plus squared exponential (custom kernel)

In [1]:
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4 (delta 1), reused 4 (delta 1), pack-reused 0
Unpacking objects: 100% (4/4), 2.36 KiB | 86.00 KiB/s, done.
From https://github.com/kenzaxtazi/nonstationary-precip
   40a5d26..cb7d1e3  main       -> origin/main
Updating 40a5d26..cb7d1e3
Fast-forward
 metrics.py |  48 +++++++++++++++++++++++
 sgpr.py    | 128 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 2 files changed, 176 insertions(+)
 create mode 100644 metrics.py
 create mode 100644 sgpr.py


In [18]:
%set_env CUDA_VISIBLE_DEVICES=0

import torch
import tqdm
import gpytorch
from gpytorch.means import ConstantMean, LinearMean
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.variational import VariationalStrategy, CholeskyVariationalDistribution
from gpytorch.distributions import MultivariateNormal
from gpytorch.models import ApproximateGP, GP
from gpytorch.mlls import VariationalELBO, AddedLossTerm
from gpytorch.likelihoods import GaussianLikelihood

env: CUDA_VISIBLE_DEVICES=0


In [19]:
from gpytorch.models.deep_gps import DeepGPLayer, DeepGP
from gpytorch.mlls import DeepApproximateMLL

In [20]:
import urllib.request
import os
from math import floor
import pandas as pd

# Load data

In [21]:
# this is for running the notebook in our testing framework
smoke_test = ('CI' in os.environ)

In [22]:
data_df = pd.read_csv('khyber_2000_2010_tp.csv')

In [23]:
if smoke_test:  # this is for running the notebook in our testing framework
    X, y = torch.randn(1000, 3), torch.randn(1000)
else:
    data = torch.Tensor(data_df.values)
    X = data[:, 1:-1]
    X = X - X.min(0)[0]
    X = 2 * (X / X.max(0)[0]) - 1
    y = data[:, -1]

In [26]:
train_n = int(floor(0.8 * len(X)))
train_x = X[:train_n, :].contiguous()
train_y = y[:train_n].contiguous()

test_x = X[train_n:, :].contiguous()
test_y = y[train_n:].contiguous()

In [27]:
if torch.cuda.is_available():
    train_x, train_y, test_x, test_y = train_x.cuda(), train_y.cuda(), test_x.cuda(), test_y.cuda()

In [28]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

## Exact GP class 

In [29]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = kernel

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [30]:
# Kernels
SE_kernel = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
custom_kernel = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=3) + gpytorch.kernels.RBFKernel(ard_num_dims=1, active_dims=[0])* gpytorch.kernels.PeriodicKernel(active_dims=[0]))

In [38]:
# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood, custom_kernel)

## Training

In [39]:
# this is for running the notebook in our testing framework
import os
smoke_test = ('CI' in os.environ)
training_iter = 2 if smoke_test else 50

In [40]:
# Find optimal model hyperparameters
model.train()
likelihood.train()

GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

In [41]:
# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

In [42]:
# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

In [44]:
for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f'   #lengthscale: %.3f   noise: %.3f' 
          % (i + 1, training_iter, loss.item(),
        #model.covar_module.base_kernel.lengthscale.item(),
        #model.likelihood.noise.item()
    ))
    optimizer.step()

Iter 1/50 - Loss: 4.940
Iter 2/50 - Loss: 4.590
Iter 3/50 - Loss: 4.336
Iter 4/50 - Loss: 4.062
Iter 5/50 - Loss: 3.662
Iter 6/50 - Loss: 3.514
Iter 7/50 - Loss: 3.337
Iter 8/50 - Loss: 3.138
Iter 9/50 - Loss: 2.887
Iter 10/50 - Loss: 2.667
Iter 11/50 - Loss: 2.562
Iter 12/50 - Loss: 2.500
Iter 13/50 - Loss: 2.424
Iter 14/50 - Loss: 2.317
Iter 15/50 - Loss: 2.206
Iter 16/50 - Loss: 2.126
Iter 17/50 - Loss: 2.079
Iter 18/50 - Loss: 1.996
Iter 19/50 - Loss: 1.953
Iter 20/50 - Loss: 1.880
Iter 21/50 - Loss: 1.844
Iter 22/50 - Loss: 1.814
Iter 23/50 - Loss: 1.793
Iter 24/50 - Loss: 1.783
Iter 25/50 - Loss: 1.782
Iter 26/50 - Loss: 1.776
Iter 27/50 - Loss: 1.778
Iter 28/50 - Loss: 1.776
Iter 29/50 - Loss: 1.771
Iter 30/50 - Loss: 1.775
Iter 31/50 - Loss: 1.772
Iter 32/50 - Loss: 1.771
Iter 33/50 - Loss: 1.772
Iter 34/50 - Loss: 1.771
Iter 35/50 - Loss: 1.772
Iter 36/50 - Loss: 1.771
Iter 37/50 - Loss: 1.771
Iter 38/50 - Loss: 1.773
Iter 39/50 - Loss: 1.771
Iter 40/50 - Loss: 1.772
Iter 41/5

## Metrics

In [49]:
def negative_log_predictive_density(test_y, predicted_mean, predicted_var):
    
    # Vector of log-predictive density per test point    
    lpd = torch.distributions.Normal(predicted_mean, torch.sqrt(predicted_var)).log_prob(test_y)
    
    # return the average
    return -torch.mean(lpd)

In [50]:
def sqrt_mean_squared_error(test_y, predicted_mean):
        
    return torch.sqrt(torch.mean((test_y - predicted_mean)**2))

In [51]:
model.eval()
with torch.no_grad():
    trained_pred_dist = likelihood(model(test_x))
    predictive_mean = trained_pred_dist.mean
    predictive_variances = trained_pred_dist.variance

rmse = sqrt_mean_squared_error(test_y, predictive_mean)
nlpd = negative_log_predictive_density(test_y, predictive_mean, predictive_variances)

print(f"RMSE: {rmse.item()}, NLPD: {nlpd.item()}")

RMSE: 3.3276002407073975, NLPD: 2.8366599082946777
